<a href="https://colab.research.google.com/github/SatioO/DL_Practices/blob/master/PSPNET_with_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [5]:
!unzip "/content/drive/My Drive/Launch pad/tfrecord.zip"

Archive:  /content/drive/My Drive/Launch pad/tfrecord.zip
replace tfrecord/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: tfrecord/.DS_Store      
  inflating: __MACOSX/tfrecord/._.DS_Store  
  inflating: tfrecord/train-00002-of-00004.tfrecord  
  inflating: tfrecord/val-00002-of-00004.tfrecord  
  inflating: tfrecord/train-00003-of-00004.tfrecord  
  inflating: tfrecord/val-00003-of-00004.tfrecord  
  inflating: tfrecord/train-00000-of-00004.tfrecord  
  inflating: tfrecord/val-00000-of-00004.tfrecord  
  inflating: tfrecord/train-00001-of-00004.tfrecord  
  inflating: tfrecord/val-00001-of-00004.tfrecord  


In [0]:
IMG_DIM = 256
BATCH_SIZE = 32
N_CLASSES = 64
OUTPUT_CHANNELS = 21
OUTPUT_DIR = "/content/tfrecord"

def normalize(input_image, input_mask):
    return input_image, input_mask
    
def parse_dataset(example_proto):
    features = {
        'image/encoded':
            tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/height':
            tf.io.FixedLenFeature((), tf.int64, default_value=0),
        'image/width':
            tf.io.FixedLenFeature((), tf.int64, default_value=0),
        'image/segmentation':
            tf.io.FixedLenFeature((), tf.string, default_value=''),
    }
    
    parsed_feature = tf.io.parse_single_example(example_proto, features)
    
    image = tf.image.decode_jpeg(parsed_feature['image/encoded'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (IMG_DIM, IMG_DIM))
    image = tf.image.per_image_standardization(image)
    
    label = tf.image.decode_png(parsed_feature['image/segmentation'], 1)
    label = tf.cast(label, tf.int32)
    
    return image, label

train_list_ds = tf.data.Dataset.list_files(OUTPUT_DIR + "/train-*.tfrecord")
valid_list_ds = tf.data.Dataset.list_files(OUTPUT_DIR + "/val-*.tfrecord")

train_ds = (tf.data
    .TFRecordDataset(train_list_ds)
    .map(parse_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE))

valid_ds = (tf.data
    .TFRecordDataset(valid_list_ds)
    .map(parse_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE))

train_dataset = train_ds.cache().shuffle(BATCH_SIZE * 50).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_ds.batch(BATCH_SIZE)

In [0]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2), dilation=1):
  # A block that has a conv layer at shortcut.
  filter1, filter2, filter3 = filters
  
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  strides = (1, 1) if dilation > 1 else strides

  x = layers.Conv2D(filter1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
  x = layers.BatchNormalization(name=bn_name_base + '2a')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filter2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b', dilation_rate=dilation)(x)
  x = layers.BatchNormalization(name=bn_name_base + '2b')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filter3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(x)
  x = layers.BatchNormalization(name=bn_name_base + '2c')(x)

  shortcut = layers.Conv2D(filter3, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input_tensor)
  shortcut = layers.BatchNormalization(name=bn_name_base + '1')(shortcut)

  x = layers.add([x, shortcut])
  x = layers.Activation('relu')(x)

  return x

def identity_block(input_tensor, kernel_size, filters, stage, block, dilation=1):
  # The identity block is the block that has no conv layer at shortcut.
  filter1, filter2, filter3 = filters

  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  x = layers.Conv2D(filter1, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2a')(input_tensor)
  x = layers.BatchNormalization(name=bn_name_base + '2a')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filter2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b', dilation_rate=dilation)(x)
  x = layers.BatchNormalization(name=bn_name_base + '2b')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filter3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(x)
  x = layers.BatchNormalization(name=bn_name_base + '2c')(x)
  x = layers.add([x, input_tensor])
  x = layers.Activation('relu')(x)

  return x


def resnet50(img_input):
  x = layers.ZeroPadding2D((3, 3), name='conv1_pad')(img_input)
  x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='valid', kernel_initializer='he_normal', name='conv1')(x)
  x = layers.BatchNormalization(name='bn_conv1')(x)
  x = layers.Activation('relu')(x)
  x = layers.ZeroPadding2D((1, 1), name='pool1_pad')(x)
  x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  f1 = x

  x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
  x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
  x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
  f2 = x

  x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
  f3 = x

  x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
  f4 = x

  x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
  x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
  x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
  f5 = x

  return [f1, f2, f3, f4, f5]


img_input = keras.layers.Input((IMG_DIM, IMG_DIM, 3))
model = resnet50(img_input)